In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time 
import requests
import random

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\Rob\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [3]:
url_df = pd.read_csv("../Resources/base_url_list.csv")

url_df
url_list = url_df["urls"] 

In [4]:
# Take list of urls generated in step one.  For each url in step 1, 
# this generates a list of pages for step 2 scraping

url_step_2_list = []

for url in url_list:
    
    browser.visit(url)
    time.sleep(random.randint(3, 6))
    
    # finds the number of pages needed to visit in this subset
    
    spans = browser.find_by_tag("span")

    for span in spans:
        if "</strong> of <strong>" in span.html:
            page_counter_html = span.html
    page_results = int(page_counter_html.split(" - ")[1].split("</strong>")[0])
    total_results = int(page_counter_html.split("of <strong>")[1].split("</strong>")[0])

    results_mod = total_results % page_results
    numpages = int(total_results / page_results)
    if results_mod != 0:
        numpages += 1

        
    # generate suffixes to add to base url
    suffixes = ['']
    for i in range(1,numpages):
        suffixes.append(f'#resultsPage={i+1}')
    suffixes
    
    for suffix in suffixes:
        url_step_2_list.append(url + suffix)

for url in url_step_2_list:
    print(url)
    print()

https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d320

https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d320#resultsPage=2

https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d320#resultsPage=3

https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d320#resultsPage=4

https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_f

In [5]:
url_step2_df = pd.DataFrame({
    'urls': url_step_2_list
})
url_step2_df.to_csv("../Resources/url_step_2.csv", index=False)